In [1]:
# === Estrai [Nome, Email, DataAcquisto] e ordina per data di acquisto =========
import os
import pandas as pd
from datetime import datetime

CSV_PATH = r"C:\Users\spina\Documents\Other_Codes\7chackras\Documenti\ticket2026.csv"

# Lettura robusta: tutto come stringhe
df = pd.read_csv(
    CSV_PATH,
    sep=",",
    quotechar='"',
    encoding="utf-8",
    engine="python",
    dtype=str,
    skip_blank_lines=True
)

def parse_payment_date(s: str):
    if s is None:
        return None
    s = str(s).replace("–", "-").replace("—", "-").strip()
    for fmt in ("%d/%m/%Y - %H:%M", "%d/%m/%Y %H:%M", "%d/%m/%Y", "%d-%m-%Y %H:%M"):
        try:
            return datetime.strptime(s, fmt)
        except ValueError:
            pass
    return None

def clean(s):
    return "" if s is None else str(s).strip()

# Colonne di riferimento
att_last_col   = "Last Name"
att_email_col  = "Attendee E-mail"
pay_date_col   = "Payment Date"
buy_first_col  = "Buyer First Name"
buy_last_col   = "Buyer Last Name"
buy_email_col  = "Buyer E-Mail"

rows = []
for _, r in df.iterrows():
    buyer_first = clean(r.get(buy_first_col))
    buyer_last  = clean(r.get(buy_last_col))
    attendee_last = clean(r.get(att_last_col))

    if buyer_first or buyer_last:
        name = (buyer_first + " " + buyer_last).strip()
    elif attendee_last:
        name = attendee_last
    else:
        name = ""

    email = clean(r.get(att_email_col)) or clean(r.get(buy_email_col))
    raw_date = clean(r.get(pay_date_col))
    dt = parse_payment_date(raw_date)
    iso = dt.isoformat(timespec="minutes") if dt else raw_date

    if not name and email:
        name = email

    rows.append({
        "Nome": name,
        "Email": email.lower(),
        "DataAcquisto": iso,
        "DataParsed": dt  # aggiungiamo colonna tecnica per ordinare
    })

out = pd.DataFrame(rows)
out = out[out["Email"].str.len() > 0].reset_index(drop=True)

# Ordina per DataParsed (più vecchio → più recente)
out = out.sort_values(by="DataParsed", ascending=True, na_position="last").reset_index(drop=True)

# Rimuovi la colonna tecnica
out = out.drop(columns=["DataParsed"])

# Salva accanto al CSV originale
out_path = os.path.join(os.path.dirname(CSV_PATH), "ticket_contacts.csv")
out.to_csv(out_path, index=False, encoding="utf-8")

print(f"Creato file ordinato per data di acquisto: {out_path}")
print(out.head(10))


Creato file ordinato per data di acquisto: C:\Users\spina\Documents\Other_Codes\7chackras\Documenti\ticket_contacts.csv
                           Nome                            Email  \
0              Andrea Sangiorgi  dott.andrea.sangiorgi@gmail.com   
1              Shamar Droghetti       shamar.droghetti@gmail.com   
2              Shamar Droghetti            p.andergassen@gmx.net   
3               Samantha de Leo        samanthadeleo@outlook.com   
4  Ivonne Rosa Daveti Vetricini       nicolas.1soldati@gmail.com   
5  Ivonne Rosa Daveti Vetricini             ciuffi.ivo@gmail.com   
6               Alessandro Bigi             btalle-92@hotmail.it   
7           Alessandro Scottini  alessandro.scottini@hotmail.com   
8                 Iryna Savchuk              iryna.dkb@gmail.com   
9                 Daniela Maier             maierda318@gmail.com   

       DataAcquisto  
0  2025-07-28T13:53  
1  2025-07-28T15:06  
2  2025-07-28T15:06  
3  2025-07-28T16:44  
4  2025-07-28T20:12  

In [2]:
# === Estrai [Nome, Email, DataAcquisto], ordina per data e salva anche per fase ===
import os
import re
import pandas as pd
from datetime import datetime

CSV_PATH = r"C:\Users\spina\Documents\Other_Codes\7chackras\Documenti\ticket2026.csv"

# Lettura robusta: tutto come stringhe
df = pd.read_csv(
    CSV_PATH,
    sep=",",
    quotechar='"',
    encoding="utf-8",
    engine="python",
    dtype=str,
    skip_blank_lines=True
)

def parse_payment_date(s: str):
    if s is None:
        return None
    s = str(s).replace("–", "-").replace("—", "-").strip()
    for fmt in ("%d/%m/%Y - %H:%M", "%d/%m/%Y %H:%M", "%d/%m/%Y", "%d-%m-%Y %H:%M"):
        try:
            return datetime.strptime(s, fmt)
        except ValueError:
            pass
    return None

def clean(s):
    return "" if s is None else str(s).strip()

# Colonne di riferimento (se mancano, il codice fa fallback sugli altri campi)
att_last_col   = "Last Name"
att_email_col  = "Attendee E-mail"
pay_date_col   = "Payment Date"
buy_first_col  = "Buyer First Name"
buy_last_col   = "Buyer Last Name"
buy_email_col  = "Buyer E-Mail"
ticket_type_col = "Ticket Type"

# Funzione per estrarre la "fase" da Ticket Type
def extract_phase(ticket_type: str) -> str:
    if not ticket_type:
        return "UNKNOWN"
    t = ticket_type.upper()
    # Cerca "PHASE <numero>"
    m = re.search(r"\bPHASE\s*([0-9]+)\b", t)
    if m:
        return f"PHASE_{m.group(1)}"
    # Gestisci Early Bird / Fellow
    if "EARLY BIRD" in t:
        return "EARLY_BIRD_FELLOW" if "FELLOW" in t else "EARLY_BIRD"
    # Nessun match esplicito
    return "OTHER"

rows = []
for _, r in df.iterrows():
    buyer_first = clean(r.get(buy_first_col))
    buyer_last  = clean(r.get(buy_last_col))
    attendee_last = clean(r.get(att_last_col))
    ticket_type = clean(r.get(ticket_type_col))
    phase = extract_phase(ticket_type)

    # Nome: preferisci nome+cognome buyer; altrimenti cognome attendee; altrimenti email
    if buyer_first or buyer_last:
        name = (buyer_first + " " + buyer_last).strip()
    elif attendee_last:
        name = attendee_last
    else:
        name = ""

    # Email: preferisci attendee, poi buyer
    email = clean(r.get(att_email_col)) or clean(r.get(buy_email_col))

    # Data acquisto: prova a parsare -> ISO, altrimenti stringa raw
    raw_date = clean(r.get(pay_date_col))
    dt = parse_payment_date(raw_date)
    iso = dt.isoformat(timespec="minutes") if dt else raw_date

    if not name and email:
        name = email

    rows.append({
        "Nome": name,
        "Email": email.lower(),
        "DataAcquisto": iso,
        "DataParsed": dt,
        "Phase": phase
    })

out = pd.DataFrame(rows)

# Rimuovi righe senza email (opzionale ma consigliato)
out = out[out["Email"].str.len() > 0].reset_index(drop=True)

# Ordina per DataParsed (più vecchio → più recente)
out = out.sort_values(by="DataParsed", ascending=True, na_position="last").reset_index(drop=True)

# Salva CSV complessivo (senza colonna tecnica)
save_dir = os.path.dirname(CSV_PATH)
all_path = os.path.join(save_dir, "ticket_contacts.csv")
out.drop(columns=["DataParsed"]).to_csv(all_path, index=False, encoding="utf-8")
print(f"Creato: {all_path}")

# Crea un CSV per ciascuna fase
def safe_slug(s: str) -> str:
    s = s.replace(" ", "_")
    return re.sub(r"[^A-Za-z0-9_\-]+", "", s)

for phase_value in sorted(out["Phase"].dropna().unique()):
    sub = out[out["Phase"] == phase_value].drop(columns=["DataParsed"]).reset_index(drop=True)
    slug = safe_slug(phase_value)
    phase_path = os.path.join(save_dir, f"ticket_contacts_{slug}.csv")
    sub.to_csv(phase_path, index=False, encoding="utf-8")
    print(f"Creato: {phase_path}  (righe: {len(sub)})")


Creato: C:\Users\spina\Documents\Other_Codes\7chackras\Documenti\ticket_contacts.csv
Creato: C:\Users\spina\Documents\Other_Codes\7chackras\Documenti\ticket_contacts_EARLY_BIRD_FELLOW.csv  (righe: 67)
Creato: C:\Users\spina\Documents\Other_Codes\7chackras\Documenti\ticket_contacts_PHASE_0.csv  (righe: 50)
Creato: C:\Users\spina\Documents\Other_Codes\7chackras\Documenti\ticket_contacts_PHASE_1.csv  (righe: 20)


In [3]:
# === Estrai contatti: Buyer vs Titolare, ordina per data e salva anche per Phase ===
import os
import re
import pandas as pd
from datetime import datetime

# Percorso del CSV
CSV_PATH = r"C:\Users\spina\Documents\Other_Codes\7chackras\Documenti\ticket2026.csv"

# Lettura robusta (tutto come stringhe)
df = pd.read_csv(
    CSV_PATH,
    sep=",",
    quotechar='"',
    encoding="utf-8",
    engine="python",
    dtype=str,
    skip_blank_lines=True
)

def parse_payment_date(s: str):
    if s is None:
        return None
    s = str(s).replace("–", "-").replace("—", "-").strip()
    for fmt in ("%d/%m/%Y - %H:%M", "%d/%m/%Y %H:%M", "%d/%m/%Y", "%d-%m-%Y %H:%M"):
        try:
            return datetime.strptime(s, fmt)
        except ValueError:
            pass
    return None

def clean(s):
    return "" if s is None else str(s).strip()

# Colonne (dall'export)
att_last_col    = "Last Name"          # titolare biglietto (solo cognome/nome libero)
att_email_col   = "Attendee E-mail"    # email titolare
buy_first_col   = "Buyer First Name"   # acquirente
buy_last_col    = "Buyer Last Name"
buy_email_col   = "Buyer E-Mail"
pay_date_col    = "Payment Date"
ticket_type_col = "Ticket Type"

# Estrai la fase dal Ticket Type
def extract_phase(ticket_type: str) -> str:
    if not ticket_type:
        return "UNKNOWN"
    t = ticket_type.upper()
    m = re.search(r"\bPHASE\s*([0-9]+)\b", t)
    if m:
        return f"PHASE_{m.group(1)}"
    if "EARLY BIRD" in t:
        return "EARLY_BIRD_FELLOW" if "FELLOW" in t else "EARLY_BIRD"
    return "OTHER"

rows = []
for _, r in df.iterrows():
    buyer_first = clean(r.get(buy_first_col))
    buyer_last  = clean(r.get(buy_last_col))
    buyer_email = clean(r.get(buy_email_col)).lower()
    holder_name = clean(r.get(att_last_col))          # “titolare del biglietto”
    holder_mail = clean(r.get(att_email_col)).lower() # email titolare
    ticket_type = clean(r.get(ticket_type_col))
    phase       = extract_phase(ticket_type)

    # Nome/Email devono corrispondere all'ACQUIRENTE (buyer)
    buyer_name = (buyer_first + " " + buyer_last).strip()
    if not buyer_name:
        # fallback prudente se i campi buyer mancano
        buyer_name = holder_name or buyer_email

    # Data acquisto
    raw_date = clean(r.get(pay_date_col))
    dt = parse_payment_date(raw_date)
    iso = dt.isoformat(timespec="minutes") if dt else raw_date

    rows.append({
        "Nome": buyer_name,                 # acquirente
        "Email": buyer_email,               # acquirente
        "TitolareBiglietto": holder_name,   # intestatario
        "EmailTitolare": holder_mail,       # email intestatario
        "DataAcquisto": iso,
        "DataParsed": dt,
        "Phase": phase
    })

out = pd.DataFrame(rows)

# Opzionale: rimuovi righe senza email acquirente
out = out[out["Email"].str.len() > 0].reset_index(drop=True)

# Ordina per data di acquisto (dal più vecchio al più recente)
out = out.sort_values(by="DataParsed", ascending=True, na_position="last").reset_index(drop=True)

# Salvataggi
save_dir = os.path.dirname(CSV_PATH)

# File completo senza la colonna tecnica
all_path = os.path.join(save_dir, "ticket_contacts.csv")
out.drop(columns=["DataParsed"]).to_csv(all_path, index=False, encoding="utf-8")
print(f"Creato: {all_path}  (righe: {len(out)})")

# Un file per ciascuna fase
def safe_slug(s: str) -> str:
    s = s.replace(" ", "_")
    return re.sub(r"[^A-Za-z0-9_\-]+", "", s)

for phase_value in sorted(out["Phase"].dropna().unique()):
    sub = out[out["Phase"] == phase_value].drop(columns=["DataParsed"]).reset_index(drop=True)
    slug = safe_slug(phase_value)
    phase_path = os.path.join(save_dir, f"ticket_contacts_{slug}.csv")
    sub.to_csv(phase_path, index=False, encoding="utf-8")
    print(f"Creato: {phase_path}  (righe: {len(sub)})")


Creato: C:\Users\spina\Documents\Other_Codes\7chackras\Documenti\ticket_contacts.csv  (righe: 137)
Creato: C:\Users\spina\Documents\Other_Codes\7chackras\Documenti\ticket_contacts_EARLY_BIRD_FELLOW.csv  (righe: 67)
Creato: C:\Users\spina\Documents\Other_Codes\7chackras\Documenti\ticket_contacts_PHASE_0.csv  (righe: 50)
Creato: C:\Users\spina\Documents\Other_Codes\7chackras\Documenti\ticket_contacts_PHASE_1.csv  (righe: 20)


In [4]:
# === Contatti ticket: acquirente completo + titolare (cognome) =================
import os
import re
import pandas as pd
from datetime import datetime

CSV_PATH = r"C:\Users\spina\Documents\Other_Codes\7chackras\Documenti\ticket2026.csv"

# Lettura robusta
df = pd.read_csv(
    CSV_PATH,
    sep=",",
    quotechar='"',
    encoding="utf-8",
    engine="python",
    dtype=str,
    skip_blank_lines=True
)

def parse_payment_date(s: str):
    if s is None:
        return None
    s = str(s).replace("–", "-").replace("—", "-").strip()
    for fmt in ("%d/%m/%Y - %H:%M", "%d/%m/%Y %H:%M", "%d/%m/%Y", "%d-%m-%Y %H:%M"):
        try:
            return datetime.strptime(s, fmt)
        except ValueError:
            pass
    return None

def clean(s):
    return "" if s is None else str(s).strip()

# Colonne note dell'export
ATT_LAST    = "Last Name"          # cognome/multi-cognome del titolare (attendee)
ATT_EMAIL   = "Attendee E-mail"
BUY_FIRST   = "Buyer First Name"
BUY_LAST    = "Buyer Last Name"
BUY_EMAIL   = "Buyer E-Mail"
PAY_DATE    = "Payment Date"
TICKET_TYPE = "Ticket Type"

def extract_phase(ticket_type: str) -> str:
    if not ticket_type:
        return "UNKNOWN"
    t = ticket_type.upper()
    m = re.search(r"\bPHASE\s*([0-9]+)\b", t)
    if m:
        return f"PHASE_{m.group(1)}"
    if "EARLY BIRD" in t:
        return "EARLY_BIRD_FELLOW" if "FELLOW" in t else "EARLY_BIRD"
    return "OTHER"

rows = []
for _, r in df.iterrows():
    buyer_first = clean(r.get(BUY_FIRST))
    buyer_last  = clean(r.get(BUY_LAST))
    buyer_email = clean(r.get(BUY_EMAIL)).lower()

    holder_cognome = clean(r.get(ATT_LAST))        # quello fornito dal CSV
    holder_email   = clean(r.get(ATT_EMAIL)).lower()

    # Nome completo acquirente
    buyer_name = (buyer_first + " " + buyer_last).strip() or buyer_email

    # Se intestatario e acquirente coincidono (stessa email), possiamo valorizzare il nome del titolare
    if holder_email and buyer_email and (holder_email == buyer_email):
        holder_nome = buyer_first
        holder_cognome_eff = buyer_last or holder_cognome
    else:
        holder_nome = ""                  # non disponibile nell'export per l'attendee
        holder_cognome_eff = holder_cognome

    raw_date = clean(r.get(PAY_DATE))
    dt = parse_payment_date(raw_date)
    iso = dt.isoformat(timespec="minutes") if dt else raw_date

    phase = extract_phase(clean(r.get(TICKET_TYPE)))

    rows.append({
        # Dati ACQUIRENTE (buyer)
        "BuyerFirstName": buyer_first,
        "BuyerLastName": buyer_last,
        "Nome": buyer_name,
        "Email": buyer_email,

        # Dati TITOLARE (attendee)
        "TitolareNome": holder_nome,
        "TitolareCognome": holder_cognome_eff,
        "Titolare": (holder_nome + " " + holder_cognome_eff).strip() if (holder_nome or holder_cognome_eff) else "",
        "EmailTitolare": holder_email,

        # Altro
        "DataAcquisto": iso,
        "DataParsed": dt,
        "Phase": phase
    })

out = pd.DataFrame(rows)

# Tieni solo righe con email acquirente
out = out[out["Email"].str.len() > 0].reset_index(drop=True)

# Ordina cronologicamente
out = out.sort_values(by="DataParsed", ascending=True, na_position="last").reset_index(drop=True)

# Salvataggi
save_dir = os.path.dirname(CSV_PATH)

# CSV completo (rimuove la colonna tecnica)
full_path = os.path.join(save_dir, "ticket_contacts.csv")
out.drop(columns=["DataParsed"]).to_csv(full_path, index=False, encoding="utf-8")
print(f"Creato: {full_path}  (righe: {len(out)})")

# CSV per ciascuna Phase
def safe_slug(s: str) -> str:
    s = s.replace(" ", "_")
    return re.sub(r"[^A-Za-z0-9_\-]+", "", s)

for phase in sorted(out["Phase"].dropna().unique()):
    sub = out[out["Phase"] == phase].drop(columns=["DataParsed"]).reset_index(drop=True)
    path = os.path.join(save_dir, f"ticket_contacts_{safe_slug(phase)}.csv")
    sub.to_csv(path, index=False, encoding="utf-8")
    print(f"Creato: {path}  (righe: {len(sub)})")


Creato: C:\Users\spina\Documents\Other_Codes\7chackras\Documenti\ticket_contacts.csv  (righe: 137)
Creato: C:\Users\spina\Documents\Other_Codes\7chackras\Documenti\ticket_contacts_EARLY_BIRD_FELLOW.csv  (righe: 67)
Creato: C:\Users\spina\Documents\Other_Codes\7chackras\Documenti\ticket_contacts_PHASE_0.csv  (righe: 50)
Creato: C:\Users\spina\Documents\Other_Codes\7chackras\Documenti\ticket_contacts_PHASE_1.csv  (righe: 20)


In [1]:
# === Contatti ticket: buyer completo + titolare (con placeholder '—' se mancano nome/cognome) ===
import os
import re
import pandas as pd
from datetime import datetime

CSV_PATH = r"C:\Users\spina\Documents\Other_Codes\7chackras\Documenti\eda_outputs_ListaTicketVenduti_12Nov25\ListaTicketVenduti_12Nov25.csv"
PLACEHOLDER = "————————————"  # simbolo da usare quando un campo del titolare è mancante

# Lettura robusta
df = pd.read_csv(
    CSV_PATH,
    sep=",",
    quotechar='"',
    encoding="utf-8",
    engine="python",
    dtype=str,
    skip_blank_lines=True
)

def parse_payment_date(s: str):
    if s is None:
        return None
    s = str(s).replace("–", "-").replace("—", "-").strip()
    for fmt in ("%d/%m/%Y - %H:%M", "%d/%m/%Y %H:%M", "%d/%m/%Y", "%d-%m-%Y %H:%M"):
        try:
            return datetime.strptime(s, fmt)
        except ValueError:
            pass
    return None

def clean(s):
    return "" if s is None else str(s).strip()

# Colonne note dell'export
ATT_LAST    = "Last Name"          # cognome/multi-cognome del titolare (attendee)
ATT_EMAIL   = "Attendee E-mail"
BUY_FIRST   = "Buyer First Name"
BUY_LAST    = "Buyer Last Name"
BUY_EMAIL   = "Buyer E-Mail"
PAY_DATE    = "Payment Date"
TICKET_TYPE = "Ticket Type"

def extract_phase(ticket_type: str) -> str:
    if not ticket_type:
        return "UNKNOWN"
    t = ticket_type.upper()
    m = re.search(r"\bPHASE\s*([0-9]+)\b", t)
    if m:
        return f"PHASE_{m.group(1)}"
    if "EARLY BIRD" in t:
        return "EARLY_BIRD_FELLOW" if "FELLOW" in t else "EARLY_BIRD"
    return "OTHER"

def safe_slug(s: str) -> str:
    s = s.replace(" ", "_")
    return re.sub(r"[^A-Za-z0-9_\-]+", "", s)

rows = []
for _, r in df.iterrows():
    buyer_first = clean(r.get(BUY_FIRST))
    buyer_last  = clean(r.get(BUY_LAST))
    buyer_email = clean(r.get(BUY_EMAIL)).lower()

    holder_cognome = clean(r.get(ATT_LAST))        # dall'export
    holder_email   = clean(r.get(ATT_EMAIL)).lower()

    # Nome completo acquirente
    buyer_name = (buyer_first + " " + buyer_last).strip() or buyer_email

    # Se intestatario e acquirente coincidono (stessa email), possiamo valorizzare il nome del titolare
    if holder_email and buyer_email and (holder_email == buyer_email):
        holder_nome_raw = buyer_first
        holder_cognome_eff_raw = buyer_last or holder_cognome
    else:
        holder_nome_raw = ""                  # non presente nell'export
        holder_cognome_eff_raw = holder_cognome

    # Applica placeholder per visibilità dei mancanti
    holder_nome = holder_nome_raw if holder_nome_raw else PLACEHOLDER
    holder_cognome_eff = holder_cognome_eff_raw if holder_cognome_eff_raw else PLACEHOLDER

    # Costruisci campo "Titolare" coerente coi placeholder:
    if holder_nome == PLACEHOLDER and holder_cognome_eff == PLACEHOLDER:
        titolare_full = PLACEHOLDER
    else:
        left = "" if holder_nome == PLACEHOLDER else holder_nome
        right = "" if holder_cognome_eff == PLACEHOLDER else holder_cognome_eff
        titolare_full = f"{left} {right}".strip() if (left or right) else PLACEHOLDER

    raw_date = clean(r.get(PAY_DATE))
    dt = parse_payment_date(raw_date)
    iso = dt.isoformat(timespec="minutes") if dt else raw_date

    phase = extract_phase(clean(r.get(TICKET_TYPE)))

    rows.append({
        # Dati ACQUIRENTE (buyer)
        "BuyerFirstName": buyer_first,
        "BuyerLastName": buyer_last,
        "Nome": buyer_name,
        "Email": buyer_email,

        # Dati TITOLARE (attendee) con placeholder
        "TitolareNome": holder_nome,
        "TitolareCognome": holder_cognome_eff,
        "Titolare": titolare_full,
        "EmailTitolare": holder_email,

        # Altro
        "DataAcquisto": iso,
        "DataParsed": dt,
        "Phase": phase
    })

out = pd.DataFrame(rows)

# Tieni solo righe con email acquirente
out = out[out["Email"].str.len() > 0].reset_index(drop=True)

# Ordina cronologicamente
out = out.sort_values(by="DataParsed", ascending=True, na_position="last").reset_index(drop=True)

# Salvataggi
save_dir = os.path.dirname(CSV_PATH)

# CSV completo (rimuove la colonna tecnica)
full_path = os.path.join(save_dir, "ticket_contacts.csv")
out.drop(columns=["DataParsed"]).to_csv(full_path, index=False, encoding="utf-8")
print(f"Creato: {full_path}  (righe: {len(out)})")

# CSV per ciascuna Phase
for phase in sorted(out["Phase"].dropna().unique()):
    sub = out[out["Phase"] == phase].drop(columns=["DataParsed"]).reset_index(drop=True)
    path = os.path.join(save_dir, f"ticket_contacts_{safe_slug(phase)}.csv")
    sub.to_csv(path, index=False, encoding="utf-8")
    print(f"Creato: {path}  (righe: {len(sub)})")


Creato: C:\Users\spina\Documents\Other_Codes\7chackras\Documenti\eda_outputs_ListaTicketVenduti_12Nov25\ticket_contacts.csv  (righe: 201)
Creato: C:\Users\spina\Documents\Other_Codes\7chackras\Documenti\eda_outputs_ListaTicketVenduti_12Nov25\ticket_contacts_EARLY_BIRD_FELLOW.csv  (righe: 88)
Creato: C:\Users\spina\Documents\Other_Codes\7chackras\Documenti\eda_outputs_ListaTicketVenduti_12Nov25\ticket_contacts_PHASE_0.csv  (righe: 73)
Creato: C:\Users\spina\Documents\Other_Codes\7chackras\Documenti\eda_outputs_ListaTicketVenduti_12Nov25\ticket_contacts_PHASE_1.csv  (righe: 40)
